# 编曲机器人

## 问题描述 

在学习生活中我们都会用到曲子，但大家用的曲子不免会重合，造成尴尬的场面。而且有时还会遇到版权问题——曲子需要付费才能使用。在网上茫茫曲海中找到一首适合自己的曲子无疑是大海捞针。那怎样才能找到一首免费的风格相近的，且独一无二的曲子呢？为此我研发了“编曲机器人”，来解决这一系列问题。

## 方案描述

1. 原曲欣赏
1. 针对原曲进行数字化处理，得到曲子的数字队列
1. 数据预处理。准备X输入变量和y变量
    1. 把曲子的数字队列切成5个一组。
    1. X输入变量：前四个数字化音符 
    1. y变量：第五个数字化音符
1. 把X变量和y变量交给模型进行训练，得到训练好的模型
    1. 为了确保模型完全学习了原曲的特征，我们定义：模型的准确度必须大于95%
1. 准备新曲子的初始曲调，对其进行数字化处理，得到数字队列
    1. 初始曲调由原曲前四个音符组成
1. 编写新的曲调
    1. 把初始曲调的数字队列输入训练好的模型，得到模型的输出，作为下一个音符，把新的音符加在曲调的最后。
    1. 再取新曲调的最后四个音符，用来预测下一个音符
    1. 重复以上步骤，得到所有音符的预测结果形成的数字队列即为机器人创造的新曲子
1. 新曲欣赏
    1. 把数字转换成音符，进行播放

## 实现过程

### 准备工作一：导入相关程序包，准备环境
* 本程序使用mingus包+fluidsynth，进行曲谱弹奏
* fluidsynth使用的soundfont: U20PIANO.SF2
* 以下程序需在windows环境下才可运行


### Import Package

In [8]:
# Windows version
import sys
import os

if sys.platform == 'win32':
    print("Add fluidsynth/bin to the path in Windoes OS")
    basepath = os.getcwd()
    dllspath = os.path.join(basepath, 'fluidsynth', 'bin')
    os.environ['PATH'] = dllspath + ';' + os.environ['PATH']
else:
    print("No action is needed")


Add fluidsynth/bin to the path in Windoes OS


In [9]:
import mingus.core.notes as notes
import mingus.core.progressions as progressions
from mingus.core import chords
from mingus.containers import NoteContainer, Note
from mingus.midi import fluidsynth

In [10]:
import pandas as pd
import numpy as np

import time

### Load the Soundfont

In [11]:
fluidsynth.init("U20PIANO.SF2")

True

### 准备工作二：静态变量及参数准备

静态变量：
* map_int_to_note ：把数字转换成音符
* map_note_to_int ：把音符转换成数字

In [12]:
map_int_to_note = pd.read_csv('int_note.csv', header=0, index_col=0).to_dict()['note']

map_note_to_int = pd.read_csv('int_note.csv', header=0, index_col=1).to_dict()['int']


In [14]:
# test
print(map_int_to_note[1])
print(map_note_to_int['D-4'])

D-4
1


## 第一步：原曲欣赏

In [16]:
#以四分音符为一拍
rhythm_per_beat = 1/4

music = [('A-3',1), ('E-4',1), ('E-4',1), ('E-4',1), ('E-4',0.5), ('D-4',0.5), ('C-4',1), ('D-4',2),
        ('D-4',1.5), ('E-4',0.5), ('D-4',1), ('C-4',1), ('D-4',0.5), ('C-4',0.5), ('A-3',0.5), ('G-3',0.5), ('A-3',2),
        ('E-3',1), ('E-3', 0.5), ('G-3',.5), ('G-3',1), ('G-3',1), ('A-3',1), ('A-3',.5), ('D-4',.5), ('C-4', 2),
        ('A-3',1), ('A-3', 0.5), ('C-4',.5), ('D-4',.5), ('E-4',.5), ('C-4',.5), ('D-4',.5), ('E-4',2), ('E-4',2)]

for i, r in music:
    n = Note(i)
    n.channel = 1
    n.velocity = 100
    fluidsynth.play_Note(n)
    time.sleep(r*rhythm_per_beat*2)

## 第二步：针对原曲进行数字化处理，得到曲子的数字队列

In [17]:
int_music = [ (map_note_to_int[x], r) for x, r in music ]
print(int_music)

[(-2, 1), (2, 1), (2, 1), (2, 1), (2, 0.5), (1, 0.5), (0, 1), (1, 2), (1, 1.5), (2, 0.5), (1, 1), (0, 1), (1, 0.5), (0, 0.5), (-2, 0.5), (-3, 0.5), (-2, 2), (-5, 1), (-5, 0.5), (-3, 0.5), (-3, 1), (-3, 1), (-2, 1), (-2, 0.5), (1, 0.5), (0, 2), (-2, 1), (-2, 0.5), (0, 0.5), (1, 0.5), (2, 0.5), (0, 0.5), (1, 0.5), (2, 2), (2, 2)]


## 第三步：数据预处理。准备X输入变量和y变量
1. 把音符的数字队列切成5个一组。
1. X输入变量：前四个数字化音符
1. y变量：第五个数字化音符

In [18]:
arr_music = np.array(int_music)
print(arr_music)

[[-2.   1. ]
 [ 2.   1. ]
 [ 2.   1. ]
 [ 2.   1. ]
 [ 2.   0.5]
 [ 1.   0.5]
 [ 0.   1. ]
 [ 1.   2. ]
 [ 1.   1.5]
 [ 2.   0.5]
 [ 1.   1. ]
 [ 0.   1. ]
 [ 1.   0.5]
 [ 0.   0.5]
 [-2.   0.5]
 [-3.   0.5]
 [-2.   2. ]
 [-5.   1. ]
 [-5.   0.5]
 [-3.   0.5]
 [-3.   1. ]
 [-3.   1. ]
 [-2.   1. ]
 [-2.   0.5]
 [ 1.   0.5]
 [ 0.   2. ]
 [-2.   1. ]
 [-2.   0.5]
 [ 0.   0.5]
 [ 1.   0.5]
 [ 2.   0.5]
 [ 0.   0.5]
 [ 1.   0.5]
 [ 2.   2. ]
 [ 2.   2. ]]


In [19]:
arr_music.shape

(35, 2)

In [20]:
arr_music[:, 0]

array([-2.,  2.,  2.,  2.,  2.,  1.,  0.,  1.,  1.,  2.,  1.,  0.,  1.,
        0., -2., -3., -2., -5., -5., -3., -3., -3., -2., -2.,  1.,  0.,
       -2., -2.,  0.,  1.,  2.,  0.,  1.,  2.,  2.])

In [21]:
# Vairable 
int_music_len = arr_music.shape[0]
interval_len = 4
arr_music_int = arr_music[:, 0]

In [22]:
dataset_list = []
for i in range(int_music_len-interval_len):
    item = arr_music_int[i:i+5]
    dataset_list.append(item)

In [ ]:
# dataset_list = [ arr_music_int[i:i+5] for i in range(int_music_len-interval_len) ]

In [ ]:
dataset_list

In [ ]:
# The item will be set as object, if the lenth of item is different
dataset = np.array(dataset_list)

In [ ]:
dataset

In [ ]:
dataset[:, 0:4]

In [ ]:
X_train, y_train = dataset[:, 0:4], dataset[:, 4]
print(X_train.shape)
print(y_train.shape)

## 第四步：把X变量和y变量交给分类模型进行训练，得到训练好的模型
1. 为了确保模型完全学习了原曲的特征，我们定义：模型的准确度必须大于95%

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
composer = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0).fit(X_train, y_train)

composer.score(X_train, y_train)

## 第五步：准备新曲子的初始曲调，对其进行数字化处理，得到数字队列
1. 初始曲调由原曲前四个音符组成

In [ ]:
init_music_notes = ['A-3', 'E-4', 'E-4', 'E-4']
new_music_int = [ map_note_to_int[x] for x in init_music_notes ]
print(new_music_int)

In [ ]:
interval_len

## 第六步：编写新的曲调
1. 把初始曲调的数字队列输入训练好的模型，得到模型的输出，作为下一个音符
1. 重复以上步骤，得到所有音符的预测结果形成的数字队列即为机器人创造的新曲子


In [ ]:
for idx in range(int_music_len-interval_len):
    next_note_int = composer.predict([new_music_int[-4:]])
    new_music_int.append(int(next_note_int))
print(new_music_int)

## 第七步：新曲欣赏
1. 把数字转换成音符，进行播放

In [ ]:
new_music_notes = [ (map_int_to_note[x], r ) for x, (_, r) in zip(new_music_int, int_music) ]
new_music_notes

In [ ]:
for i, r in new_music_notes:
    n = Note(i)
    n.channel = 1
    n.velocity = 100
    fluidsynth.play_Note(n)
    time.sleep(r*rhythm_per_beat*2)

## 结论

### 方案体会

我们已经做到了让机器人学习原始曲调，并且根据原始曲调创建新的、与原曲有着相似风格的曲调达到了我们制作此机器人的目的。

### 问题及优化

曲子的差异还比较明显。我们还要结合音乐的特性对机器人进行优化。